In [100]:
import pandas as pd
import numpy as np
import random
import itertools

'''
code for testing 

NOT IMPORTANT!

'''
np.random.seed(37)
random.seed(37)

height = [
    np.random.normal(5.5, 1.0, 100),
    np.random.normal(5.3, 1.0, 200),
    np.random.normal(5.9, 1.0, 300),
    np.random.normal(5.7, 1.0, 200),
    np.random.normal(5.3, 1.0, 400),
    np.random.normal(5.2, 1.0, 500),
    np.random.normal(5.8, 1.0, 300),
    np.random.normal(5.5, 1.0, 200)
]

#height = normal distribution **(HAVE TO BE FETCHED FROM PUMS) 

demographic = [
    ['white', 'minor', 'male'],
    ['white', 'minor', 'female'],
    ['white', 'adult', 'male'],
    ['white', 'adult', 'female'],
    ['other', 'minor', 'male'],
    ['other', 'minor', 'female'],
    ['other', 'adult', 'male'],
    ['other', 'adult', 'female']
]

data = [[{'race': d[0], 'age': d[1], 'gender': d[2], 'height': h} for h in s]
        for d, s in zip(demographic, height)]
data = list(itertools.chain(*data))

df = pd.DataFrame(data)
df.head()

,race,age,gender,height
0,white,minor,male,5.445536
1,white,minor,male,6.174308
2,white,minor,male,5.846647
3,white,minor,male,4.199654
4,white,minor,male,7.018512


#### demographic = cell names for ipf
#### df = generated from cleaned pums data
 

In [101]:
#import mapped data headers [pums:nhts]
races=['white','black','native','asian','pacific-islander','other']
genders=['male','female']
ages=['child','tenager','young-adult','adult','senior']
educ=['below-high','High-school','college','Bachelor','Graduate']
incomecat=['1','2','3','4','5','6','7','8','9','10','11','0']
vehs=['1','2','3','0']
nps=['2', '1', '4', '3', '5', '6', '7', '8', '9', '11', '10', '15', '20', '17', '16', '12', '14']

# demographic = [[i, j, k, l, m, a ,b] for i in races for j in ages for k in genders for l in educ for m in incomecat for a in vehs for b in np]
demographic = list(itertools.product(ages,genders,races,educ))
demographic
# demographic ready. 

# data = pums

[('child', 'male', 'white', 'below-high'),
 ('child', 'male', 'white', 'High-school'),
 ('child', 'male', 'white', 'college'),
 ('child', 'male', 'white', 'Bachelor'),
 ('child', 'male', 'white', 'Graduate'),
 ('child', 'male', 'black', 'below-high'),
 ('child', 'male', 'black', 'High-school'),
 ('child', 'male', 'black', 'college'),
 ('child', 'male', 'black', 'Bachelor'),
 ('child', 'male', 'black', 'Graduate'),
 ('child', 'male', 'native', 'below-high'),
 ('child', 'male', 'native', 'High-school'),
 ('child', 'male', 'native', 'college'),
 ('child', 'male', 'native', 'Bachelor'),
 ('child', 'male', 'native', 'Graduate'),
 ('child', 'male', 'asian', 'below-high'),
 ('child', 'male', 'asian', 'High-school'),
 ('child', 'male', 'asian', 'college'),
 ('child', 'male', 'asian', 'Bachelor'),
 ('child', 'male', 'asian', 'Graduate'),
 ('child', 'male', 'pacific-islander', 'below-high'),
 ('child', 'male', 'pacific-islander', 'High-school'),
 ('child', 'male', 'pacific-islander', 'college'),

In [102]:
col_list = ["SERIALNO","SEX", "RAC1P", "AGEP","SCHL","HINCP","VEH","NP","WIF"]
# col_list = ["SEX", "RAC1P", "AGEP","SCHL"]
df = pd.read_csv("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\CA-data\\pums\\temp_pums.csv", usecols=col_list)

df.head(5)
df=df.fillna(df.median())
df.isnull().sum()

C:\Users\abubabu\AppData\Local\Temp\ipykernel_31052\748897660.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df=df.fillna(df.median())


SERIALNO    0
AGEP        0
SCHL        0
SEX         0
RAC1P       0
HINCP       0
VEH         0
NP          0
WIF         0
dtype: int64

#### cell 66-73 for visualization

In [103]:
df.RAC1P.value_counts().sort_index() / df.shape[0]

asian               0.007746
black               0.010470
native              0.099027
other               0.038575
pacific-islander    0.000513
white               0.843669
Name: RAC1P, dtype: float64

In [104]:
df.AGEP.value_counts().sort_index() / df.shape[0]

adult          0.385392
child          0.074002
senior         0.286276
tenager        0.175909
young-adult    0.078422
Name: AGEP, dtype: float64

In [105]:
df.SEX.value_counts().sort_index() / df.shape[0]

female    0.496071
male      0.503929
Name: SEX, dtype: float64

#### marginal functions

In [106]:
def get_target_marginals(d):
    factors = list(d.keys())
    targets = [sorted([(k2, v2) for k2, v2 in v.items()]) for k, v in d.items()]
    targets = np.array([[v for _, v in item] for item in targets])
    # print("inside func",type(targets))
    return factors, targets

def get_table(df, targets):
    factors, target_marginals = get_target_marginals(targets)

    cross_tab = pd.crosstab(df[factors[0]], [df[c] for c in factors[1:]], dropna=False)
    # pd.set_option('display.max_columns', None)
    # with open('cross_tabulation.txt', 'w') as file:
    #     file.write(str(cross_tab))
    # cross_tab = pd.crosstab(df[factors[0]], [df[factors[1]],df[factors[2]],df[factors[3]]])
    shape = tuple([df[c].unique().shape[0] for c in factors])
    print(shape)

    table = cross_tab.values.reshape(shape)
    
    return factors, target_marginals, table


In [107]:
# # factors = list(df.keys())
# factors=['AGEP','SCHL','RAC1P','SEX']
# cross_tab = pd.crosstab(df[factors[0]], [df[c] for c in factors[1:]], dropna=False)
# # cross_tab = df.pivot_table(index=factors[0], columns=[c for c in factors[1:]], aggfunc=len, fill_value=0)
# cross_tab.values.shape
# # factors
# # df.isnull().sum()
df['NP'].unique()

array([ 3.,  2.,  4.,  1.,  5., 10.,  7.,  6.,  8., 15.,  9., 11., 20.,
       12., 17., 14., 16.])

#### table = contigency table

Loading data from census

In [108]:
#ddg = demographic_data_groups
ddg=pd.read_csv("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\CA-data\\census\\temp_demographic.csv")
edg=pd.read_csv("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\CA-data\\census\\temp_economic.csv")
hdg=pd.read_csv("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\CA-data\\census\\temp_housing.csv")
sdg=pd.read_csv("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\CA-data\\census\\temp_social.csv")
edudg=pd.read_csv("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\CA-data\\census\\temp_educational.csv")


#### fetching margin values from demographic data groups 

In [109]:
temp_race = {'white': 'DP05_0064E', 'black': 'DP05_0065E', 'native': 'DP05_0066E',
             'asian': 'DP05_0067E', 'pacific-islander': 'DP05_0068E', 'other': 'DP05_0069E'}
temp_sex = {'male': 'DP05_0002E', 'female': 'DP05_0003E'}
temp_age = {
    'child': ddg.loc[ddg['id'] == 'DP05_0005E', 'value'].values[0],
    'tenager': ddg.loc[ddg['id'] == 'DP05_0006E', 'value'].values[0]+ddg.loc[ddg['id'] == 'DP05_0007E', 'value'].values[0]+ddg.loc[ddg['id'] == 'DP05_0008E', 'value'].values[0],
    'young-adult': ddg.loc[ddg['id'] == 'DP05_0009E', 'value'].values[0],
    'adult': ddg.loc[ddg['id'] == 'DP05_0010E', 'value'].values[0]+ddg.loc[ddg['id'] == 'DP05_0011E', 'value'].values[0]+ddg.loc[ddg['id'] == 'DP05_0012E', 'value'].values[0]+ddg.loc[ddg['id'] == 'DP05_0013E', 'value'].values[0],
    'senior': ddg.loc[ddg['id'] == 'DP05_0014E', 'value'].values[0]+ddg.loc[ddg['id'] == 'DP05_0015E', 'value'].values[0]+ddg.loc[ddg['id'] == 'DP05_0016E', 'value'].values[0]+ddg.loc[ddg['id'] == 'DP05_0017E', 'value'].values[0]
}
temp_inc = {
    '1': int(edg.loc[edg['id'] == 'DP03_0076E', 'value'].values[0]),
    '2': int(edg.loc[edg['id'] == 'DP03_0077E', 'value'].values[0]),
    '3': int(edg.loc[edg['id'] == 'DP03_0078E', 'value'].values[0]),
    '4': int(edg.loc[edg['id'] == 'DP03_0079E', 'value'].values[0]),
    '5': int(edg.loc[edg['id'] == 'DP03_0080E', 'value'].values[0]),
    '6': int(edg.loc[edg['id'] == 'DP03_0081E', 'value'].values[0]),
    '7': int(edg.loc[edg['id'] == 'DP03_0082E', 'value'].values[0]),
    '8': int(edg.loc[edg['id'] == 'DP03_0083E', 'value'].values[0]),
    '9': int(edg.loc[edg['id'] == 'DP03_0084E', 'value'].values[0]),
    '10': int(edg.loc[edg['id'] == 'DP03_0085E', 'value'].values[0]),
    '0': int(edg.loc[edg['id'] == 'DP03_0076E', 'value'].values[0])

}
temp_veh={
    '1' : int(hdg.loc[hdg['id'] == 'DP04_0059E', 'value'].values[0]),
    '2' : int(hdg.loc[hdg['id'] == 'DP04_0060E', 'value'].values[0]),
    '3' : int(hdg.loc[hdg['id'] == 'DP04_0061E', 'value'].values[0]),
    '0' : int(hdg.loc[hdg['id'] == 'DP04_0058E', 'value'].values[0]),
}

temp_np={ #counted from pums
    '2': 13904,
    '1': 7603,
    '4': 7276,
    '3': 5979,
    '5': 5170,
    '6': 2328,
    '7': 1036,
    '8': 424,
    '9': 288,
    '11': 143,
    '10': 120,
    '15': 45,
    '20': 40,
    '17': 34,
    '16': 32,
    '12': 24,
    '14': 14
}
temp_educ={
    'below-high': int(edudg.loc[edudg['id'] == 'S1501_C01_002E', 'value'].values[0])+int(edudg.loc[edudg['id'] == 'S1501_C01_007E', 'value'].values[0])+int(edudg.loc[edudg['id'] == 'S1501_C01_008E', 'value'].values[0]),
    'High-school':int(edudg.loc[edudg['id'] == 'S1501_C01_003E', 'value'].values[0])+int(edudg.loc[edudg['id'] == 'S1501_C01_009E', 'value'].values[0])+int(edudg.loc[edudg['id'] == 'S1501_C01_010E', 'value'].values[0]),
    'college':int(edudg.loc[edudg['id'] == 'S1501_C01_004E', 'value'].values[0])+int(edudg.loc[edudg['id'] == 'S1501_C01_011E', 'value'].values[0]),
    'Bachelor':int(edudg.loc[edudg['id'] == 'S1501_C01_005E', 'value'].values[0])+int(edudg.loc[edudg['id'] == 'S1501_C01_012E', 'value'].values[0]),
    'Graduate':int(edudg.loc[edudg['id'] == 'S1501_C01_013E', 'value'].values[0])
}
margin_dict = {
    'SEX': {key: ddg.loc[ddg['id'] == value, 'value'].values[0] for key, value in temp_sex.items()},
    'AGEP': temp_age,
    "SCHL":temp_educ,
    'RAC1P': {key: ddg.loc[ddg['id'] == value, 'value'].values[0] for key, value in temp_race.items()},
    'HINCP':temp_inc,
    'VEH': temp_veh,
    'NP' : temp_np
}
# ddg.columns
# df = pd.read_csv("CA-data\\pums\\temp_pums.csv", usecols=col_list)
margin_dict


{'SEX': {'male': 19526298.0, 'female': 19757199.0},
 'AGEP': {'child': 2451528.0,
  'tenager': 7608859.0,
  'young-adult': 2751567.0,
  'adult': 18748401.0,
  'senior': 7723142.0},
 'SCHL': {'below-high': 30337897,
  'High-school': 114149312,
  'college': 32431968,
  'Bachelor': 47075168,
  'Graduate': 27274058},
 'RAC1P': {'white': 25048942.0,
  'black': 2766478.0,
  'native': 762733.0,
  'asian': 6551732.0,
  'pacific-islander': 323042.0,
  'other': 5944874.0},
 'HINCP': {'1': 296174,
  '2': 200509,
  '3': 547788,
  '4': 613545,
  '5': 907148,
  '6': 1378391,
  '7': 1156498,
  '8': 1638022,
  '9': 924788,
  '10': 1295573,
  '0': 296174},
 'VEH': {'1': 3968129, '2': 4851748, '3': 3296432, '0': 927957},
 'NP': {'2': 13904,
  '1': 7603,
  '4': 7276,
  '3': 5979,
  '5': 5170,
  '6': 2328,
  '7': 1036,
  '8': 424,
  '9': 288,
  '11': 143,
  '10': 120,
  '15': 45,
  '20': 40,
  '17': 34,
  '16': 32,
  '12': 24,
  '14': 14}}

In [110]:
# df["HINCP"].unique().shape[0]

In [111]:
f, u, X = get_table(df, margin_dict)

C:\Users\abubabu\AppData\Local\Temp\ipykernel_31052\2856549065.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  targets = np.array([[v for _, v in item] for item in targets])


(2, 5, 5, 6, 11, 4, 17)


In [112]:
X.shape

(2, 5, 5, 6, 11, 4, 17)

In [113]:
type(X)

numpy.ndarray

In [114]:
for i in u:
    i=np.array(i)
    print(type(i))
    for j in i :
        print(type(j))

<class 'numpy.ndarray'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.ndarray'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.ndarray'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.ndarray'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.ndarray'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.ndarray'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.ndarray'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.

X = contigency table\
u = target marginals\
f = helper vector

In [115]:
X

array([[[[[[[  0,   0,   0, ...,   0,   0,   0],
            [  0,   0,   0, ...,   0,   0,   0],
            [  0,   0,   0, ...,   0,   0,   0],
            [  0,   0,   0, ...,   0,   0,   0]],

           [[  0,   0,   0, ...,   0,   0,   0],
            [  0,   0,   0, ...,   0,   0,   0],
            [  0,   0,   0, ...,   0,   0,   0],
            [  0,   0,   0, ...,   0,   0,   0]],

           [[  0,   0,   0, ...,   0,   0,   0],
            [  0,   0,   1, ...,   0,   0,   0],
            [  0,   0,   0, ...,   0,   0,   0],
            [  0,   0,   0, ...,   0,   0,   0]],

           ...,

           [[  0,   0,   0, ...,   0,   0,   0],
            [  0,   1,   0, ...,   0,   0,   0],
            [  0,   1,   0, ...,   0,   0,   0],
            [  0,   1,   0, ...,   0,   0,   0]],

           [[  0,   0,   0, ...,   0,   0,   0],
            [  0,   0,   0, ...,   0,   0,   0],
            [  0,   0,   0, ...,   0,   0,   0],
            [  0,   0,   0, ...,   0,   0,  

In [116]:
X=np.nan_to_num(X, nan=0.0).astype('float64')

In [117]:
def get_coordinates(M):
    return list(itertools.product(*[list(range(n)) for n in M.shape]))

# returns a dict with marginals for each unique value of dimention i in M
def get_marginals(M, i):
    coordinates = get_coordinates(M)
    #print(M)  # M hoilo marginals
    key = lambda tup: tup[0]
    counts = [(c[i], M[c]) for c in coordinates]
    counts = sorted(counts, key=key)
    counts = itertools.groupby(counts, key=key)
    counts = {k: sum([v[1] for v in g]) for k, g in counts}

    return counts

def get_all_marginals(M):
    return np.array([[v for _, v in get_marginals(M, i).items()]
                     for i in range(len(M.shape))])

# returns a dict with counts for each unique value of dimention i in M
def get_counts(M, i):
    coordinates = get_coordinates(M)

    key = lambda tup: tup[0]
    counts = [(c[i], M[c], c) for c in coordinates]
    counts = sorted(counts, key=key)
    counts = itertools.groupby(counts, key=key)
    counts = {k: [(tup[1], tup[2]) for tup in g] for k, g in counts}

    return counts

#ipu to update values 
def update_values(M, i, u):
    marg = get_marginals(M, i)
    vals = get_counts(M, i)
    # print("u -> ",type(u) )
    # print("marg ->",type(marg))
    # for k, v in vals.items():
    #     for n, c in v:
    #         if not isinstance(u[k], np.float64):
    #             print(f"u[k] is not of type numpy.float64 in key '{k}'",type(u[k]), print(u[k]))
            
    #         if not isinstance(marg[k], np.int64):
    #             print(f"marg[k] is not of type numpy.int64 in key '{k}'")
    #         # print("n ->", type(n))
    #         # print("u[k] ->", type(u[k]))
    #         # print("marg[k] ->", type(marg[k]))
    d = [[(c, n * u[k] / marg[k]) for n, c in v] for k, v in vals.items()]
    d = itertools.chain(*d)
    d = list(d)

    return d

def ipf_update(M, u):
    for i in range(len(M.shape)):
        values = update_values(M, i, u[i])
        for idx, v in values:
            # print(idx)
            M[idx] = v

    o = get_all_marginals(M)
    d = get_deltas(o, u)

    return M, d

# o-t
def get_deltas(o, t):
    return np.array([np.linalg.norm(np.array(o[r]) - np.array(t[r]), 2) for r in range(o.shape[0])])


def get_weights(X, max_iters=50, zero_threshold=0.0001, convergence_threshold=3, debug=True):
    M = X.copy()

    d_prev = np.zeros(len(M.shape))
    count_zero = 0

    for _ in range(max_iters):
        print(_,count_zero)
        # print(u)
        M, d_next = ipf_update(M, u)
        d = np.linalg.norm(d_prev - d_next, 2)

        if d < zero_threshold:
            count_zero += 1

        if debug:
            print(','.join([f'{v:.5f}' for v in d_next]), d)
        d_prev = d_next

        # breaks if zero threshold triggered in 3 consecutive run
        if count_zero >= convergence_threshold:
            break

    w = M / M.sum()
    return w

In [118]:
w = get_weights(X)

0 0


C:\Users\abubabu\AppData\Local\Temp\ipykernel_31052\1557694917.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([[v for _, v in get_marginals(M, i).items()]


27746638.97654,21944884.01052,133999821.71730,26703293.97390,3173590.54336,7117860.56003,0.00000 141354899.64604887
1 0
27746677.21489,21940680.52670,133999953.83351,26694911.96001,3172419.07463,7118245.55058,0.00000 9458.694767133542
2 0
27746671.88579,21942111.63191,134000252.60493,26695729.36086,3172660.20234,7117939.80376,0.00000 1719.6286617092435
3 0
27746669.42558,21942568.74748,134000020.64667,26696157.94274,3172835.76257,7117793.19005,0.00000 706.2327811401075
4 0
27746667.55429,21942803.87333,133999865.36873,26696439.59689,3172965.01020,7117692.76459,0.00000 430.7182254848818
5 0
27746666.35268,21942940.19924,133999768.52037,26696625.34764,3173051.95445,7117625.55468,0.00000 273.03042436723746
6 0
27746665.73646,21943030.20905,133999709.85659,26696751.97939,3173107.34914,7117581.84006,0.00000 180.4403641421748
7 0
27746665.50880,21943101.60232,133999678.56530,26696843.05020,3173142.07905,7117553.30773,0.00000 128.0248004158487
8 0
27746665.49741,21943164.57463,133999664.83889

In [119]:
w

array([[[[[[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
             0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
            [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
             0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
            [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
             0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
            [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
             0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

           [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
             0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
            [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
             0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
            [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
             0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
            [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
             0.00000000e+00, 0.00000

In [120]:
w.shape

(2, 5, 5, 6, 11, 4, 17)

In [121]:
# races=['white','black','native','asian','pacific-islander','other']
# genders=['male','female']
# ages=['child','tenager','young-adult','adult','senior']

In [122]:
# import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# array_3d=w
# # Assuming you have a numpy array called 'data' with shape (6, 5, 2)
# # Replace 'data' with the actual name of your array
# column_names = ['male','female']

# # Create row and column index labels
# row_labels = ['white','black','native','asian','pacific-islander','other']
# column_labels = ['child','tenager','young-adult','adult','senior']

# # Create multi-index for rows and columns
# row_index = pd.MultiIndex.from_product([row_labels, column_labels], names=['Race', 'Age'])
# column_index = pd.MultiIndex.from_product([column_names], names=['sex'])

# # Create DataFrame
# weight_df = pd.DataFrame(array_3d.reshape((-1, array_3d.shape[2])), index=row_index, columns=column_index)

# # Display the DataFrame
# weight_df

In [123]:
# weight_df.shape

In [124]:
# data=df
# fig, ax = plt.subplots()
# colors = np.random.rand(*data.shape)
# # Create a table from the dataframe
# table = ax.table(cellText=data, cellColours=colors, loc='center')

# # Set the font size and cell padding
# table.set_fontsize(14)
# table.scale(1, 1.5)  # Adjust the scaling if needed
# table.auto_set_column_width(col=list(range(len(data[0]))))  # Auto adjust column width

# # Hide the axes
# ax.axis('off')

# # Show the plot
# plt.show()

In [125]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# # Assuming you have a 3D DataFrame named "df"
# # Create a sample 3D DataFrame
# row_labels = ['white','black','native','asian','pacific-islander','other']
# column_labels = ['child','tenager','young-adult','adult','senior']
# value_columns = ['male','female']
# # data = np.random.rand(6, 5, 2)
# # df = pd.DataFrame(data, index=row_labels, columns=column_labels, dtype=float)
# # Plot each depth slice as a heatmap
# fig, axs = plt.subplots(1, df.shape[2], figsize=(12, 4))
# for i in range(df.shape[2]):
#     heatmap = axs[i].imshow(df.iloc[:, :, i], cmap='viridis')
#     axs[i].set_title(f'Depth {i}')
#     axs[i].set_xlabel('Column')
#     axs[i].set_ylabel('Row')
#     plt.colorbar(heatmap, ax=axs[i])

# plt.tight_layout()
# plt.show()

<b> Sampling </b>

In [126]:
import functools

def get_sampling_weights(df, f, w):
    get_filters = lambda df, fields, values: [df[f] == v for f, v in zip(fields, values)]
    get_total = lambda df, fields, values: df[functools.reduce(lambda a, b: a & b, get_filters(df, fields, values))].shape[0]

    return {k: v / get_total(df, f, k) for k, v in zip(list(itertools.product(*[sorted(df[c].unique()) for c in f])), np.ravel(w))}

def get_samples(df, f, w, n=10000):
    weights = get_sampling_weights(df, f, w)
    s = df.apply(lambda r: weights[tuple([r[c] for c in f])], axis=1)
    return df.sample(n=n, replace=True, weights=s)

sample_df = get_samples(df, f, w)
sample_df

C:\Users\abubabu\AppData\Local\Temp\ipykernel_31052\1234085552.py:7: RuntimeWarning: invalid value encountered in double_scalars
  return {k: v / get_total(df, f, k) for k, v in zip(list(itertools.product(*[sorted(df[c].unique()) for c in f])), np.ravel(w))}


,SERIALNO,AGEP,SCHL,SEX,RAC1P,HINCP,VEH,NP,WIF
33966,2020HU0852554,senior,Bachelor,male,white,10,1.0,9.0,3.0
38421,2021HU0343140,senior,High-school,male,black,6,2.0,11.0,2.0
34809,2020HU0966702,tenager,High-school,female,native,7,2.0,14.0,3.0
2031,2017000342692,adult,High-school,female,white,7,2.0,9.0,2.0
34809,2020HU0966702,tenager,High-school,female,native,7,2.0,14.0,3.0
...,...,...,...,...,...,...,...,...,...
42494,2021HU1041986,adult,Graduate,female,other,9,1.0,1.0,2.0
18767,2019HU0071859,adult,Bachelor,female,white,8,2.0,11.0,3.0
864,2017000138947,adult,Bachelor,male,asian,4,1.0,1.0,2.0
1479,2017000247487,senior,High-school,male,native,10,1.0,15.0,3.0


In [127]:
sample_df.to_csv("sampled_DATA.csv")

In [128]:
f

['SEX', 'AGEP', 'SCHL', 'RAC1P', 'HINCP', 'VEH', 'NP']

Cross-tabulation of resulting sampled matrix

In [ ]:
sample_df.to_csv("SAMPLE_DATA.csv",header=True)
#education

marginals remain kind of the same